In [1]:
import torch
from  torch.utils.data import DataLoader

In [ ]:
    

from dsetsG import LunaDataset, LunaDataset_Train, LunaDataset_Val
from test.model import LunaModel


class TrainingLoop:
    def __init__(self, model, optimizer, loss_fn, train_loader, val_loader):
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.train_loader = train_loader
        self.val_loader = val_loader

    def train(self, epochs):
        for epoch in range(epochs):
            self.model.train()
            for batch in self.train_loader:
                self.optimizer.zero_grad()
                x, y = batch
                y_pred = self.model(x)
                loss = self.loss_fn(y_pred, y)
                loss.backward()
                self.optimizer.step()

            self.model.eval()
            with torch.no_grad():
                val_loss = sum(self.loss_fn(self.model(x), y) for x, y in self.val_loader)
            print(f"Epoch {epoch}, val_loss: {val_loss}")
            
def build_training_loop():
    model = LunaModel()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    batch_size = 64
    num_workers = 4
    # dataset = LunaDataset()
    trainset = LunaDataset_Train()
    valset = LunaDataset_Val()
    # dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)
    train_loader = DataLoader(trainset, batch_size=batch_size, num_workers=num_workers)
    val_loader = DataLoader(valset, batch_size=batch_size, num_workers=num_workers)
    return TrainingLoop(model, optimizer, loss_fn, train_loader, val_loader)
def run_training_loop(training_loop, epochs):
    training_loop.train(epochs)